In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os

driver = webdriver.Chrome('c:/chromedriver.exe')
# url = 'file:///C:/Users/Playdata/Desktop/shinhancard_20200910_900046432/shinhancard_20200910_900046432.html'
url = 'file:///C:/Users/Playdata/Desktop/shinhancard_20201110_900046432/shinhancard_20201110_900046432.html'
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html,'html.parser')

tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

for n in range(len(tables_middleSum )) :
    tables.remove(tables_middleSum[n])

# 명세서에서 추출한 데이터 string으로 변환하여 분리
seperate_tables = []                          
for n in range(len(tables)) :
    table = str(tables[n].text).split('\n')
    seperate_tables = seperate_tables + table

# seperate_tables에서 '사용날짜' 추출   
date_list = []
for n in range(1,len(seperate_tables), 12) :
    date = seperate_tables[n]
    date_list.append(date)

# seperate_tables에서 '사용처' 데이터만 추출    
store_list = []
for n in range(3,len(seperate_tables),12) :
    store = seperate_tables[n].replace(" ", "")
    store_list.append(store)

# 카카오 API활용하여 seperate_tables에서 '사용처 주소' 데이터만 추출    
location_list = []
def create_url(search_text):
    headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}

    base = "https://dapi.kakao.com/v2/local/search/keyword.json"
    param_query = "?query=" +search_text
#     print(f'base + param_query: {base + param_query}')
    respond = requests.get(base + param_query,headers=headers)
    return respond.json()

for n in range(len(store_list)) :
# for n in range(10) :
    store_name = store_list[n]
    result = create_url(store_name)
    tmp1 = result['documents']
    try :
        tmp2 = tmp1[0]
        tmp3 = tmp2['address_name']
        location_list.append(tmp3)
    except :
        tmp2 = '해당 주소가 없습니다.'
        location_list.append(tmp2)

# seperate_tables에서 '사용금액' 데이터만 추출  
price_list = [] 
for n in range(4,len(seperate_tables),12) :
    price = seperate_tables[n]
    price_list.append(price)

# 추출한 데이터로 DataFrame 변환
table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
df1 = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])

driver.close()

df1

,사용날짜,사용처,사용처주소,사용금액
0,20.09.27,서호김밥,서울 서초구 방배동 790-13,"14,000"
1,20.09.27,GS25성수포레점,서울 성동구 성수동1가 685-696,"10,000"
2,20.09.28,왕창할인마트,서울 동작구 사당동 127-9,"5,700"
3,20.10.02,주식회사세녹코리아,해당 주소가 없습니다.,"5,500"
4,20.10.02,주식회사아성다이소,경남 창원시 성산구 상남동 26-4,"6,000"
...,...,...,...,...
84,20.10.22,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
85,20.10.23,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
86,20.10.23,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
87,20.10.24,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"


In [4]:
# DataFrame Excel로변환
excel_writer = pd.ExcelWriter('C:/Users/Playdata/Desktop/carduselist.xlsx', 
                              engine='xlsxwriter')
df1.to_excel(excel_writer, index = False, sheet_name='카드사용리스트_09월')
excel_writer.save()